<a href="https://colab.research.google.com/github/PontiiPilates/dostavka-backend/blob/main/telegram_groups_parser_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon

from datetime import datetime, timezone
from telethon import TelegramClient
from google.colab import ai

import asyncio
import csv

## Доступы

In [2]:
api_id = ''
api_hash = ''

## Каналы

In [3]:
channels = [
    't.me/airsiberia',
    't.me/krassea_alliance',
    't.me/shirnind',
    't.me/supturkrsk',
    # 't.me/clubalga24/3', # Анонсы ПВД (однодневных походов) # (чат)
    't.me/photo_trips', # Фотопоходы- Тропами предков! 👣
    't.me/mo_tour', # Своя тропа * Походы * Путешествия * Приключения
    't.me/geoturizm', # Геотуризм - походы и экскурсии Красноярск
    't.me/mystolby', # Мои Столбы
    't.me/na_opyte_tur', # На опыте! Cнаряжение и туризм. Акции, скидки, рекомендации.
    't.me/umberi_travel', # Клуб путешественников УмБери 🌏
    ]

channels = [
    't.me/geoturizm', # Геотуризм - походы и экскурсии Красноярск
    ]

# print(channels)

## Клиент и авторизация

In [4]:
client = TelegramClient('getContent', api_id, api_hash)

In [ ]:
# вход в систему / авторизация
await client.start()

## Скрипт для сборки Json

In [ ]:
from os import link

posts = []

for channel in channels:
    messages = await client.get_messages(channel, limit=20)

    for message in messages:

        messageDate = message.date

        # c 1 февраля
        compareDate = datetime(2026, 1, 31, 17, 0, tzinfo=timezone.utc)

        # от нужной даты
        if messageDate >= compareDate:
            # только посты
            if message.post and message.text:

                posts.append(
                    {
                        "date": f"{messageDate}",
                        "channel": channel,
                        "channel_id": message.peer_id.channel_id,
                        "post_id": message.id,
                        "link": f"https://{channel}/{message.id}",
                        "content": message.text,
                    }
                )
print(posts)

## Скрипт для сборки .csv

In [33]:
from os import link

# Открываем файл на перезапись и записываем заголовки только один раз в начале
with open("messages.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["date", "channel", "channel_id", "post_id", "link", "content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

# Теперь в цикле открываем файл в режиме добавления
for channel in channels:
    messages = await client.get_messages(channel, limit=20)

    # Открытие файла csv для добавления данных
    with open("messages.csv", "a", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["date", "channel", "channel_id", "post_id", "link", "content"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Запись каждого сообщения в CSV-файл
        for message in messages:

            messageDate = message.date

            # c 1 февраля
            compareDate = datetime(2026, 1, 31, 17, 0, tzinfo=timezone.utc)

            # от нужной даты
            if messageDate >= compareDate:
                # только посты
                if message.post and message.text:

                    writer.writerow(
                        {
                            "date": messageDate,
                            "channel": channel,
                            "channel_id": message.peer_id.channel_id,
                            "post_id": message.id,
                            "link": f"https://{channel}/{message.id}",
                            "content": message.text,
                        }
                    )

## AI обработка

In [ ]:
response = ai.generate_text("Чем закончилась история с 3I Atlas?", model_name='google/gemini-2.5-flash')
print(response)

## Завершение работы

In [ ]:
# закрытие соединения
await client.disconnect()

## Пайплайн

1. Выгружаю сообщения из каналов
  - нельзя указаь дату -> зато можно отфильтровать после выгрузки
  - выгружать нужно регулярно -> либо от заданной даты
  - раз в день по 10 постов со всех каналов -> 10 мало, изображения тоже считаются
  - проверить, есть ли дата в посте -> конечно
  - также получить ссылку на сообщение -> готово

2. Обработка AI
  - преобразование в структуру -> преобразовал
  - структура должна содержать дату поста в котором обнаружен анонс -> есть
  - теперь нужно очистить посты, которые не анонс

3. Формирование базы
  - при добавлении записей алгоритм сравнивает даты постов
  - добавляет анонс, если дата его выхода позже


TODO:
- [] выявнить, есть ли возможность выгружать сообщения из супергрупп
- [] выгрузку необходимо дополнить датой поста



## Структура ответа на метод `get_messages` (для постов)

```
{
  "messages": [
    {
      "_type": "Message",
      "id": 2075,
      "peer_id": {
        "_type": "PeerChannel",
        "channel_id": 1648212419
      },
      "date": "2026-02-05T06:00:22Z",
      "message": "🔹БИРЮСИНСКИЙ ЗАЛИВ\n(лёд Красноярского моря)                         \n\nБУДНИ \n10 февраля (вт) в 10.00\n11 февраля (ср) в 10.00\n12 февраля (чт) в 10.00\n13 февраля (пт) в 10.00\n\nВЫХОДНЫЕ\n14 февраля (сб) в 10.00                                 \n15 февраля (вс) в 10.00\n \n❄️Путешествие на Красноярское море в «Бирюсинский залив»\n\n🧊Мамонтова пещера\n🧊Царские ворота\n🧊Ущелье грот \n🔥Прожарка на костре \n\n❄️Поход по скальным объектам залива, пройдём в Мамонтову пещеру, поднимемся на Царские ворота! Посетим вертикальный  грот, прокатимся на качелях, покушаем в кемпинге с костром 🔥\n\n🚐Выезд: 10.00 Предмостная, Красный Яр. Матросова,4\nРасстояние – залив Шумиха 60км\nПитание: перекус-еда, чай-кофе!\nТеплая одежда, сидушка, ледоступы, гамаши, фонарик!\n****************************\n💗Участие: 2000р (доставка, сопровождение) \n\n®️Регистрация в личные\n\nгруппа в МАХ 📱\nгруппа в ВК 📱",
      "out": false,
      "mentioned": false,
      "media_unread": false,
      "silent": false,
      "post": true,
      "from_scheduled": false,
      "legacy": false,
      "edit_hide": false,
      "pinned": false,
      "noforwards": false,
      "invert_media": false,
      "offline": false,
      "video_processing_pending": false,
      "paid_suggested_post_stars": false,
      "paid_suggested_post_ton": false,
      "from_id": null,
      "from_boosts_applied": null,
      "saved_peer_id": null,
      "fwd_from": null,
      "via_bot_id": null,
      "via_business_bot_id": null,
      "reply_to": null,
      "media": {
        "_type": "MessageMediaDocument",
        "nopremium": false,
        "spoiler": false,
        "video": true,
        "round": false,
        "voice": false,
        "document": {
          "_type": "Document",
          "id": 5197512556805131270,
          "access_hash": -110495474101961571,
          "file_reference": "02623db9c30000081b6985f0710f5f9e2097e4e4c2e3cf678997006108",
          "date": "2026-02-05T06:00:22Z",
          "mime_type": "video/mp4",
          "size": 14316579,
          "dc_id": 2,
          "attributes": [
            {
              "_type": "DocumentAttributeVideo",
              "duration": 68.734001159668,
              "w": 848,
              "h": 448,
              "round_message": false,
              "supports_streaming": true,
              "nosound": false,
              "preload_prefix_size": null,
              "video_start_ts": null,
              "video_codec": null
            },
            {
              "_type": "DocumentAttributeFilename",
              "file_name": "IMG_3516.MOV"
            }
          ],
          "thumbs": [
            {
              "_type": "PhotoStrippedSize",
              "type": "i",
              "bytes": "0115289aea74b6f9442bcb920e3bf6a6c57ef22280ab903e6e2a4d454b59be3b60d47a74286db76ce5b3c93di8086f25c91b01c0cf4a44b86691dbe307e4c038ab66142c58asb76f5ed514f1ac568e428c85f5a684fc858c19d1491b403927d68a2c4936885ba9cff3a2902b8fb81be1e1653d0ff8d25a8d8b0228ec28a29813523a86054f4a28a403628c4518452481eb45145303f"
            },
            {
              "_type": "PhotoSize",
              "type": "m",
              "w": 320,
              "h": 169,
              "size": 14238
            }
          ],
          "video_thumbs": []
        },
        "alt_documents": [],
        "video_cover": null,
        "video_timestamp": null,
        "ttl_seconds": null
      },
      "reply_markup": null,
      "entities": [
        {
          "_type": "MessageEntityItalic",
          "offset": 20,
          "length": 49
        },
        {
          "_type": "MessageEntityBlockquote",
          "offset": 71,
          "length": 193,
          "collapsed": false
        },
        {
          "_type": "MessageEntityItalic",
          "offset": 71,
          "length": 10
        },
        {
          "_type": "MessageEntityItalic",
          "offset": 183,
          "length": 4
        },
        {
          "_type": "MessageEntityBold",
          "offset": 187,
          "length": 77
        },
        {
          "_type": "MessageEntityBold",
          "offset": 745,
          "length": 30
        },
        {
          "_type": "MessageEntityTextUrl",
          "offset": 844,
          "length": 15,
          "url": "https://max.ru/join/0m_s0bcYq41YLYxVKIvKMZKUvThQ4Uru0JLIKS5Dck0"
        },
        {
          "_type": "MessageEntityTextUrl",
          "offset": 860,
          "length": 11,
          "url": "https://vk.com/geoturizm?from=groups"
        },
        {
          "_type": "MessageEntityCustomEmoji",
          "offset": 872,
          "length": 2,
          "document_id": 5334853932915114338
        }
      ],
      "views": 471,
      "forwards": 6,
      "replies": {
        "_type": "MessageReplies",
        "replies": 19,
        "replies_pts": 42056,
        "comments": true,
        "recent_repliers": [
          {
            "_type": "PeerChannel",
            "channel_id": 1648212419
          },
          {
            "_type": "PeerUser",
            "user_id": 1930334798
          }
        ],
        "channel_id": 1803719418,
        "max_id": 25819,
        "read_max_id": null
      },
      "edit_date": "2026-02-05T07:51:36Z",
      "post_author": null,
      "grouped_id": null,
      "reactions": {
        "_type": "MessageReactions",
        "results": [
          {
            "_type": "ReactionCount",
            "reaction": {
              "_type": "ReactionEmoji",
              "emoticon": "🔥"
            },
            "count": 6,
            "chosen_order": null
          },
          {
            "_type": "ReactionCount",
            "reaction": {
              "_type": "ReactionEmoji",
              "emoticon": "👏"
            },
            "count": 4,
            "chosen_order": null
          },
          {
            "_type": "ReactionCount",
            "reaction": {
              "_type": "ReactionEmoji",
              "emoticon": "❤"
            },
            "count": 2,
            "chosen_order": null
          },
          {
            "_type": "ReactionCount",
            "reaction": {
              "_type": "ReactionEmoji",
              "emoticon": "💘"
            },
            "count": 2,
            "chosen_order": null
          }
        ],
        "min": false,
        "can_see_list": false,
        "reactions_as_tags": false,
        "recent_reactions": [],
        "top_reactors": []
      },
      "restriction_reason": [],
      "ttl_period": null,
      "quick_reply_shortcut_id": null,
      "effect": null,
      "factcheck": null,
      "report_delivery_until_date": null,
      "paid_message_stars": null,
      "suggested_post": null
    }
  ],
  "total": 1544
}
```


## Структура ответа `get_messages` (для фото)

```
{
  "id": 2070,
  "peer_id": {
    "channel_id": 1648212419,
    "_": "PeerChannel"
  },
  "date": "2026-02-02T06:49:41+00:00",
  "message": "",
  "out": false,
  "mentioned": false,
  "media_unread": false,
  "silent": false,
  "post": true,
  "from_scheduled": false,
  "legacy": false,
  "edit_hide": true,
  "pinned": false,
  "noforwards": false,
  "invert_media": false,
  "offline": false,
  "video_processing_pending": false,
  "paid_suggested_post_stars": false,
  "paid_suggested_post_ton": false,
  "from_id": null,
  "from_boosts_applied": null,
  "saved_peer_id": null,
  "fwd_from": null,
  "via_bot_id": null,
  "via_business_bot_id": null,
  "reply_to": null,
  "media": {
    "photo": {
      "id": 5188615931943391794,
      "access_hash": -8260592546971161773,
      "date": "2026-02-02T06:49:33+00:00",
      "sizes": [
        {
          "type": "i",
          "bytes": "AR4ov4Iy..."
        },
        {
          "type": "m",
          "w": 320,
          "h": 242,
          "size": 39875
        },
        {
          "type": "x",
          "w": 800,
          "h": 604,
          "size": 234666
        },
        {
          "type": "y",
          "w": 1195,
          "h": 902,
          "sizes": [
            16567,
            52876,
            109579,
            187771,
            339732
          ]
        }
      ],
      "dc_id": 2,
      "has_stickers": false
    },
    "_": "MessageMediaPhoto"
  },
  "reply_markup": null,
  "entities": [],
  "views": 533,
  "forwards": 16,
  "replies": null,
  "edit_date": "2026-02-02T06:49:44+00:00",
  "post_author": null,
  "grouped_id": 14160119848625890,
  "reactions": null,
  "restriction_reason": [],
  "ttl_period": null,
  "quick_reply_shortcut_id": null,
  "effect": null,
  "factcheck": null,
  "report_delivery_until_date": null,
  "paid_message_stars": null,
  "suggested_post": null
}
```

